# DEMO of Antakia on California Housing dataset 
### let's load the dataset and shuffle it

In [16]:
import pandas as pd
from antakia.utils.examples import fetch_dataset
df = fetch_dataset('california_housing')
df.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:e3a51e4d8cf64a0e86dfeaf58ecc69edf25...
1,size 5901911


In [ ]:
df = df.sample(len(df))
limit = int(20000 / 0.8)
df = df.iloc[:limit]

### Then split it into train and test

In [ ]:
split_row = int(len(df) * 0.8)
df_train = df[:split_row]
df_test = df[split_row:]

In [ ]:
X_train = df_train.iloc[:,:8] # the dataset
y_train = df_train.iloc[:,9] # the target variable
shap_values_train = df_train.iloc[:,[10,11,12,13,14,15,16,17]] # the SHAP values from a previous model

X_test = df_test.iloc[:,:8] # the dataset
y_test = df_test.iloc[:,9] # the target variable

### and train the model

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
regressor = GradientBoostingRegressor(random_state = 9)
regressor.fit(X_train, y_train)

### Let's explore the model

let's add some context

In [ ]:
variables_df = pd.DataFrame(
    {'col_index': [0, 1, 2, 3, 4, 5, 6, 7],
    'descr': ['Median income', 'House age', 'Average nb rooms', 'Average nb bedrooms', 'Population', 'Average occupancy', 'Latitude', 'Longitude'],
    'type': ['float64', 'int', 'float64', 'float64', 'int', 'float64', 'float64', 'float64'],
    'unit': ['k$', 'years', 'rooms', 'rooms', 'people', 'ratio', 'degrees', 'degrees'],
    'critical': [True, False, False, False, False, False, False, False],
    'lat': [False, False, False, False, False, False, True, False],
    'lon': [False, False, False, False, False, False, False, True]},
    index=['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
)

We call AntakIA passing :
1. the train dataset, 
2. the model
3. the test dataset (Optional)
4. the already computed SHAP values (Optional)
5. a description of X variables

Here is the bare minimum to run AntakIA : 
```python
atk = AntakIA(X, y, regressor)
```

In [ ]:
from antakia.antakia import AntakIA

atk = AntakIA(
    X_train, y_train,
    regressor,
    variables=variables_df,
    X_test=X_test, y_test=y_test,
    X_exp=shap_values_train
)

In [ ]:
atk.start_gui()